In [7]:
# !which google-chrome

/usr/bin/google-chrome


In [6]:
# !wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# !sudo dpkg -i google-chrome-stable_current_amd64.deb
# !sudo apt install --fix-broken -y

--2024-12-28 02:53:57--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 74.125.196.136, 74.125.196.91, 74.125.196.93, ...
Connecting to dl.google.com (dl.google.com)|74.125.196.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112770956 (108M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 107.55M   239MB/s    in 0.4s    

2024-12-28 02:53:57 (239 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [112770956/112770956]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (131.0.6778.204-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package l

In [5]:
!pip install selenium -i https://pypi.org/simple
!pip install webdriver-manager -i https://pypi.org/simple

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Inputan Kota Asal - Kota Tujuan dan Tanggal Berangkat

In [3]:
# Function to generate URL for train ticket search on Traveloka
def generate_traveloka_url(origin, destination, date):
    # Mapping kota ke kode Traveloka (sesuaikan dengan kode Traveloka yang benar)
    city_codes = {
        "jakarta": "JKT-A",
        "surabaya": "SBY-A",
        "bandung": "BDG-A",
        "yogyakarta": "YOG-A",
        "semarang": "SMG-A",
        "malang": "MLG-A",
        "solo": "SOO-A",
        "kediri": "KDR-A",
        "pasuruan": "PSR-A",
        "probolinggo": "PBL-A",
        "jember": "JMB-A",
    }

    # Pastikan kota ada dalam mapping
    origin_code = city_codes.get(origin.lower(), None)
    destination_code = city_codes.get(destination.lower(), None)

    if not origin_code or not destination_code:
        raise ValueError("Kode kota asal atau tujuan tidak ditemukan. Silakan tambahkan ke mapping `city_codes`.")

    # Format URL
    base_url = "https://www.traveloka.com/id-id/kereta-api/search"
    url = f"{base_url}?st={origin_code}.{destination_code}&dt={date}.null&ps=1.0&pd=KAI"
    return url

# Input untuk pengguna
origin = input("Masukkan kota asal: ")
destination = input("Masukkan kota tujuan: ")
date = input("Masukkan tanggal keberangkatan (DD-MM-YYYY): ")

try:
    # Generate dan cetak URL
    url = generate_traveloka_url(origin, destination, date)
    print("URL pencarian tiket kereta api:", url)
    print("diharapkan untuk memvalidasi link yang di generate")
except ValueError as e:
    print(e)


Masukkan kota asal: surabaya
Masukkan kota tujuan: jakarta
Masukkan tanggal keberangkatan (DD-MM-YYYY): 02-01-2025
URL pencarian tiket kereta api: https://www.traveloka.com/id-id/kereta-api/search?st=SBY-A.JKT-A&dt=02-01-2025.null&ps=1.0&pd=KAI
diharapkan untuk memvalidasi link yang di generate


# WebScraping dengan Selenium

In [4]:
# Konfigurasi Selenium
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Gunakan ChromeDriverManager untuk mengatur driver
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL target
driver.get(url)

# Data untuk menyimpan hasil scraping
data = []
valid_data_count = 0
max_data_count = 25 # Jumlah data yang diinginkan

# Inisialisasi WebDriverWait
wait = WebDriverWait(driver, 20)

# Scroll untuk memuat lebih banyak data
scroll_pause_time = 2
previous_height = driver.execute_script("return document.body.scrollHeight")

while valid_data_count < max_data_count:
    # Scroll ke bawah halaman
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Ambil data tiket kereta
    trains = driver.find_elements(By.XPATH, "//div[contains(@class, 'css-1dbjc4n r-14lw9ot r-kdyh1x r-1yos0t3 r-1ifxtd0 r-1udh08x r-1pcd2l5 r-kzbkwu')]")
    for train in trains:
        try:
            # Ambil elemen data
            name_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n')]/h3[contains(@class, 'css-4rbku5 css-901oao')]")
            train_name = name_element[0].text.strip() if name_element else None

            class_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n')]/div[contains(@class, 'css-901oao r-a5wbuh r-majxgm r-1cwl3u0 r-fdjqy7')]")
            train_class = class_element[0].text.strip() if class_element else None

            departure_time_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-lgvlli r-1abyd5x')]/h3")
            departure_time = departure_time_element[0].text.strip() if departure_time_element else None

            arrival_time_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-lgvlli r-1qhn6m8')]/h3")
            arrival_time = arrival_time_element[0].text.strip() if arrival_time_element else None

            departure_station_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-lgvlli r-1abyd5x')]/div")
            departure_station = departure_station_element[0].text.strip() if departure_station_element else None

            arrival_station_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-lgvlli r-1qhn6m8')]/div")
            arrival_station = arrival_station_element[0].text.strip() if arrival_station_element else None

            duration_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-1z12117 r-1ljd8xs r-dta0w2 r-labphf r-puj83k')]/h3")
            duration = duration_element[0].text.strip() if duration_element else None

            info_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-1z12117 r-1ljd8xs r-dta0w2 r-labphf r-puj83k')]/div")
            info = info_element[0].text.strip() if info_element else None

            seat_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-eqz5dr')]/div[contains(@class, 'css-901oao r-a5wbuh r-majxgm r-1cwl3u0 r-fdjqy7')]")
            seat = seat_element[0].text.strip() if seat_element else None

            price_element = train.find_elements(By.XPATH, ".//div[contains(@class, 'css-1dbjc4n r-eqz5dr')]/h2")
            price = price_element[0].text.strip() if price_element else None

            # Validasi data
            if all([train_name, train_class, departure_time, arrival_time, departure_station, arrival_station, duration, info, price]):
                data.append({
                    "Nama Kereta": train_name,
                    "Kelas Kereta": train_class,
                    "Waktu Berangkat": departure_time,
                    "Waktu Tiba": arrival_time,
                    "Stasiun Awal": departure_station,
                    "Stasiun Tiba": arrival_station,
                    "Waktu Perjalanan": duration,
                    "Info Transit": info,
                    "Kursi Tersisa": seat,
                    "Harga Tiket": price
                })
                valid_data_count += 1

                # Berhenti jika jumlah data mencapai maksimum
                if valid_data_count >= max_data_count:
                    break

        except Exception as e:
            print(f"Error extracting train data: {e}")
            continue

    # Cek apakah ada lebih banyak data untuk dimuat
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == previous_height:
        print("Tidak ada data baru yang dimuat.")
        break
    previous_height = new_height

# Simpan data ke DataFrame
df = pd.DataFrame(data, columns=["Nama Kereta", "Kelas Kereta", "Waktu Berangkat", "Waktu Tiba", "Stasiun Awal", "Stasiun Tiba", "Waktu Perjalanan", "Info Transit", "Kursi Tersisa", "Harga Tiket"])

# Tutup driver
driver.quit()

df

Error extracting train data: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.204); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x57b4d19fb8fa <unknown>
#1 0x57b4d150cd20 <unknown>
#2 0x57b4d1512ae3 <unknown>
#3 0x57b4d1514fa8 <unknown>
#4 0x57b4d1515033 <unknown>
#5 0x57b4d155b6c5 <unknown>
#6 0x57b4d155bd01 <unknown>
#7 0x57b4d15505f3 <unknown>
#8 0x57b4d157fb1d <unknown>
#9 0x57b4d15500a8 <unknown>
#10 0x57b4d157fcbe <unknown>
#11 0x57b4d159e560 <unknown>
#12 0x57b4d157f893 <unknown>
#13 0x57b4d154e30d <unknown>
#14 0x57b4d154f32e <unknown>
#15 0x57b4d19c800b <unknown>
#16 0x57b4d19cbf97 <unknown>
#17 0x57b4d19b471c <unknown>
#18 0x57b4d19ccb17 <unknown>
#19 0x57b4d19996cf <unknown>
#20 0x57b4d19ea6b8 <unknown>
#21 0x57b4d19ea880 <unknown>
#22 0x57b4d19fa776 <unknown>
#23 0x7afc96f5aac3 <unknown>


,Nama Kereta,Kelas Kereta,Waktu Berangkat,Waktu Tiba,Stasiun Awal,Stasiun Tiba,Waktu Perjalanan,Info Transit,Kursi Tersisa,Harga Tiket
0,Sembrani Tambahan (7007A),Eksekutif (AA),05:45,15:27,Surabaya Pasar Turi,Jatinegara,9j 42m,Langsung,,Rp 654.000
1,Sembrani Tambahan (7007A),Eksekutif (AA),05:45,15:45,Surabaya Pasar Turi,Gambir,10j 0m,Langsung,,Rp 654.000
2,Sembrani (61),Eksekutif (AA),08:00,16:17,Surabaya Pasar Turi,Jatinegara,8j 17m,Langsung,Sisa 21 kursi,Rp 800.000
3,Sembrani (61),Eksekutif (AA),08:00,16:34,Surabaya Pasar Turi,Gambir,8j 34m,Langsung,Sisa 21 kursi,Rp 800.000
4,Sembrani (61),Eksekutif (AB),08:00,16:17,Surabaya Pasar Turi,Jatinegara,8j 17m,Langsung,,Rp 820.000
5,Sembrani (61),Eksekutif (AB),08:00,16:34,Surabaya Pasar Turi,Gambir,8j 34m,Langsung,,Rp 820.000
6,Sembrani Luxury (61L),Eksekutif (H),08:00,16:17,Surabaya Pasar Turi,Jatinegara,8j 17m,Langsung,Sisa 22 kursi,Rp 1.300.000
7,Sembrani Luxury (61L),Eksekutif (H),08:00,16:34,Surabaya Pasar Turi,Gambir,8j 34m,Langsung,Sisa 22 kursi,Rp 1.300.000
8,Sembrani Luxury (61L),Eksekutif (A),08:00,16:17,Surabaya Pasar Turi,Jatinegara,8j 17m,Langsung,Sisa 25 kursi,Rp 1.540.000
9,Sembrani Luxury (61L),Eksekutif (A),08:00,16:34,Surabaya Pasar Turi,Gambir,8j 34m,Langsung,Sisa 25 kursi,Rp 1.540.000


Terdapat output error dikarenakan ada element yang tidak di ambil oleh selenium dan itu tidak mempengaruhi data yang akan menjadi dataframe karena sudah di lakukan validasi sehingga data yang tidak di ambil akan di buang dan di gantikan data yang baru


data yang tidak di ambil ini adalah data dengan element yang tidak terbaca oleh selenium tetapi ke aslian data asli masih tetap terjaga